In [146]:
# inportar librerias
import sqlite3  # Módulo para trabajar con bases de datos SQLite3
import pandas as pd  # Módulo para trabajar con estructuras de datos tipo DataFrame

# conectarse a la base de datos
conn = sqlite3.connect("data/db_movies")
cur = conn.cursor()  # Crear un cursor para ejecutar sentencias SQL

# consultamos las tablas de la base de datos
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('ratings',), ('movies',), ('movies_clean',), ('full_table',), ('data_clean',)]


In [147]:
# ver tabla de data_clean
df = pd.read_sql_query("SELECT * FROM data_clean", conn)
df.head()

,movieId,title,year,userId,rating,timestamp,Action,Adventure,Animation,Children,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995,1,4.0,964982703,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,Toy Story (1995),1995,5,4.0,847434962,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,Toy Story (1995),1995,7,4.5,1106635946,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,Toy Story (1995),1995,15,2.5,1510577970,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,Toy Story (1995),1995,17,4.5,1305696483,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


## FILTRO BASADO EN POPULARIDAD

### TOP 2 PELÍCULAS MEJOR CALIFICADAS POR AÑO

In [59]:
############################# las 2 mejores calificados por año ###################
cal_year = pd.read_sql(
    """select year, title, 
            avg(iif(rating = 0, Null, rating)) as avg_rat,
            count(iif(rating = 0, Null, rating)) as rat_numb,
            count(*) as read_num
            from data_clean
            group by  year, title
            order by year desc, avg_rat desc
            """,
    conn,
)

# Quitar columna rat_numb y read_num
cal_year = cal_year.drop(["rat_numb", "read_num"], axis=1)

# Aproximar a 2 decimales
cal_year = cal_year.round(1)

# Escoger de la tabla anterior 2 peliculas por año con mejor calificación pero mostrando todos los años
cal_year = cal_year.groupby("year").head(2)


# Borrar la columna year
cal_year = cal_year.drop(["year"], axis=1)

# Visualizar
cal_year

,title,avg_rat
0,Zootopia (2016),3.9
1,Deadpool (2016),3.8
2,The Martian (2015),4.0
3,Kingsman: The Secret Service (2015),4.0
8,Guardians of the Galaxy (2014),4.1
...,...,...
856,Fantasia (1940),3.8
857,Pinocchio (1940),3.5
858,"Wizard of Oz, The (1939)",3.9
859,Gone with the Wind (1939),3.6


In [60]:
# Función pidiendo el año
def cal_year(year):
    cal_year = pd.read_sql(
        """select year, title, 
            avg(iif(rating = 0, Null, rating)) as avg_rat,
            count(iif(rating = 0, Null, rating)) as rat_numb,
            count(*) as read_num
            from data_clean
            group by  year, title
            order by year desc, avg_rat desc
            """,
        conn,
    )

    # Quitar columna rat_numb y read_num
    cal_year = cal_year.drop(["rat_numb", "read_num"], axis=1)

    # Aproximar a 2 decimales
    cal_year = cal_year.round(1)

    # Escoger de la tabla anterior 2 peliculas por año con mejor calificación pero mostrando todos los años
    cal_year = cal_year.groupby("year").head(2)

    return cal_year[cal_year["year"] == year]


cal_year("2003")

,year,title,avg_rat
190,2003,"Lord of the Rings: The Return of the King, The...",4.1
191,2003,Old Boy (2003),4.1


### TOP 1 PELÍCULAS POR GÉNERO

In [154]:
# Mejores películas por género
# (nota: Film noir y Sci fi nos da problemas)

lista = [
    "Action",
    "Adventure",
    "Animation",
    "Children",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    # "Film-Noir",
    "Horror",
    "IMAX",
    "Musical",
    "Mystery",
    "Romance",
    # "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]

for i in lista:
    cal_action = pd.read_sql(
        f"""select title, 
                avg(iif(rating = 0, Null, rating)) as avg_rat,
                count(iif(rating = 0, Null, rating)) as rat_numb,
                count(*) as read_num
                from data_clean
                where {i} = 1
                group by  title
                order by avg_rat desc limit 1
                """,
        conn,
    )

    best_genres = cal_action["title"].tolist()

    print(f"La mejor pelicula de {i} es: {best_genres[0]}")

La mejor pelicula de Action es: Fight Club (1999)
La mejor pelicula de Adventure es: Lawrence of Arabia (1962)
La mejor pelicula de Animation es: Spirited Away (Sen to Chihiro no kamikakushi) (2001)
La mejor pelicula de Children es: Toy Story 3 (2010)
La mejor pelicula de Comedy es: Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
La mejor pelicula de Crime es: Shawshank Redemption, The (1994)
La mejor pelicula de Documentary es: Roger & Me (1989)
La mejor pelicula de Drama es: Shawshank Redemption, The (1994)
La mejor pelicula de Fantasy es: Princess Bride, The (1987)
La mejor pelicula de Horror es: Silence of the Lambs, The (1991)
La mejor pelicula de IMAX es: Dark Knight, The (2008)
La mejor pelicula de Musical es: Singin' in the Rain (1952)
La mejor pelicula de Mystery es: Chinatown (1974)
La mejor pelicula de Romance es: Princess Bride, The (1987)
La mejor pelicula de Thriller es: Chinatown (1974)
La mejor pelicula de War es: Lawrence of Arabia (1962)
La

### TOP 10 PELICULAS MEJOR CALIFICADAS

In [156]:
##### recomendaciones basado en popularidad ######
mej_cal = pd.read_sql(
    """select title, 
            avg(rating) as avg_rat,
            count(*) as view_num
            from data_clean
            where rating<>0
            group by title
            order by avg_rat desc
            limit 10
            """,
    conn,
)
mej_cal
# Quitar la columna avg_rat y view_num
mej_cal = mej_cal.drop(["avg_rat", "view_num"], axis=1)
mej_cal

,title
0,"Shawshank Redemption, The (1994)"
1,Lawrence of Arabia (1962)
2,"Godfather, The (1972)"
3,Dr. Strangelove or: How I Learned to Stop Worr...
4,Patton (1970)
5,Cool Hand Luke (1967)
6,Chinatown (1974)
7,Rear Window (1954)
8,Fight Club (1999)
9,"Princess Bride, The (1987)"


### TOP 10 PELÍCULAS MÁS VISTAS

In [157]:
#### las mas vistas con promedio de los que calficaron ###
mas_vis = pd.read_sql(
    """select title, 
            avg(iif(rating = 0, Null, rating)) as avg_rat,
            count(*) as view_num
            from data_clean
            group by title
            order by view_num desc
            """,
    conn,
)

mas_vis.head(15)

# Quitar la columna avg_rat y view_num
mas_vis = mas_vis.drop(["avg_rat", "view_num"], axis=1)
mas_vis

,title
0,Forrest Gump (1994)
1,"Shawshank Redemption, The (1994)"
2,Pulp Fiction (1994)
3,"Matrix, The (1999)"
4,"Silence of the Lambs, The (1991)"
...,...
856,"Producers, The (1968)"
857,JFK (1991)
858,Bio-Dome (1996)
859,Down Periscope (1996)
